#  Poppy-Right-Arm - Simulator version


The Poppy Right Arm is a project that is focused on modeling the effects of toy interaction on infant development.
For example, if given a rattle to shake, how does an infant develop motor skill such as rhythm? Are there any effects on an infants reactionss to sounds in general, and perhaps is there any effect on early language development? All of these questions are relevent to this project.

This version of the notebook is for working with the V-rep simulator.
A scene for the poppy right arm has not been developed yet, so using the stationary poppy humanoid is needed in its place.
As the physical robot can only have sensor data collected from the joints via sensors on the motor, this is useful for modeling the specifics of the hand rather than the rist in spatial relativity tothe head.


## Imports and general setup

In [ ]:
# needed for plotting within a notebook cell
%pylab inline

import time    # library for time based functionality

import numpy as np    # Python's linear algrabra library - needed for static arrays 

# Matplotlib is one of Python's standard library for plotting data
from mpl_toolkits.mplot3d import Axes3D    # This first library is specifically for plotting on a 3D plane
import matplotlib.pyplot as plt    # This is the general matplotlib library

# These are needed for the actual robot
import pypot.robot    # This is needed for the physical robot
from poppy.creatures import PoppyHumanoid    # The humanoid library is needed in order to use the simulator

poppy = PoppyHumanoid(simulator='vrep')    # initialization for the robot in the simulator

# Preps the motors for use
for m in poppy.motors:
    m.compliant = False    # makes all the joints in the robot stiff, which is better for holding positions
                           # making them flexible is for more human-robot interactions
    m.goto_behavior = 'minjerk'    # minimizes jerking motions - makes movements as smooth as possible
                                   # Folows Hogan and Flash's minimum jerk hypothesis
                                   # For more information,see: http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/MinimumJerkHypothesis.ipynb

            
# Python (array) lists to store feedback data
pos_Head = []    # Stores the position of the hand (wrist) relative to the head
pos_Stand = []    # Stores the position of the hand (wrist) relative to the center point between the feet
TIME = []    # Stores timestamps of when the arm is in a certain position
sys_load = []    # Stores the torque the motors are going through at a point in time.
                 # Good for being warry of the system load of a task and possibly calculating fatigue
speed = []    # Stores the angle speed the motors are travelling at
temp = []   # Stores the motors' temperature
volt = []   # Stores the motor's voltage. 
            # In conjunction with Temperaure, this can be used to measure energy of a task

# Motors

poppy.r_shoulder_x:
    * id: 52
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: -90.0 to 0.0 degrees 

poppy.r_shoulder_y
    * id: 51
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: -50.0 to 170.0 degrees 
    
poppy.r_arm_z
    * id: 53
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: -20.0 to 95.0 degrees 

poppy.r_elbow_y
    * id: 54
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: 0.0 to 130.0 degrees 

## A number of function commands for the robot

In [ ]:
def rest_position():
    """
    This command function returns the arm to a "resting" position, although it is more on the lines of the 
    attention position.
    
    For the physical arm, it is important to calibrate the motors so that the resting position is set for all the
    motors is at angle 0. The simulator defaults to this, so this will eliminate the need to constantly 
    translate angles when switching between the two.
    """
    poppy.r_shoulder_x.goal_position = 0   
    poppy.r_shoulder_y.goal_position = 0
    poppy.r_arm_z.goal_position = 0
    poppy.r_elbow_y.goal_position = 0
    

In [ ]:
def app():
    """
    This function merely appends system sensory information into the appropriate python list
    """
    
    # for the simulator only
    pos_Head.append(poppy.get_object_position('r_forearm_visual','head_visual')) # appends wrist position relative to head position
    pos_Stand.append(poppy.get_object_position('r_forearm_visual')) # appends wrist position relative to the center point between the feet
    
    # appends the stress each motor is going through
    sys_load.append(poppy.r_shoulder_x.present_load)
    sys_load.append(poppy.r_shoulder_y.present_load)
    sys_load.append(poppy.r_arm_z.present_load)
    sys_load.append(poppy.r_elbow_y.present_load)
   
    # appends the angle speed each motor is currently traveling at 
    speed.append(poppy.r_shoulder_x.present_speed)
    speed.append(poppy.r_shoulder_y.present_speed)
    speed.append(poppy.r_arm_z.present_speed)
    speed.append(poppy.r_elbow_y.present_speed)
    
    # appends system temperature for each motor
    temp.append(poppy.r_shoulder_x.present_temperature)
    temp.append(poppy.r_shoulder_y.present_temperature)
    temp.append(poppy.r_arm_z.present_temperature)
    temp.append(poppy.r_elbow_y.present_temperature)
    
    # appends the voltage ruuning through each motor
    volt.append(poppy.r_shoulder_x.present_voltage)
    volt.append(poppy.r_shoulder_y.present_voltage)
    volt.append(poppy.r_arm_z.present_voltage)
    volt.append(poppy.r_elbow_y.present_voltage)

In [ ]:
def hand_wave(sec=10, rest=0.5):
    """
    This function is a primitive. When called, the robotic arm will move between two set point in space, defined by the angles the motors are travelling to.
    This will make the arm perform a wave motion.
    
        :param int sec: time in seconds for how long the arm will wave
            ::default is set to 10 seconds
        :param float rest: time in seconds for the time the arm has to finish its motion
            ::default is set to 0.5 seconds
    """
    
    t0 = time.time()   # Gets the timestamp at the start of the waving motion
    
    
    # This will loop through the two arm positions until the set time duration has elapsed. 
    while True:

        # arm moves to first position
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = 30
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -135
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = -120;
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = 0
        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)

        
        # arm moves to second position
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = 45
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -175
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = -120
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = -65
        
        app()
        TIME.append(t1-t0)
        
        time.sleep(rest)
        app()
        TIME.append(t1-t0)
        
        
        t1 = time.time() # Gets the timestamp at the end of each loop
        
        if t1-t0 >= sec: # This will end the loop and wave motion when the allotted time has expired
            break

In [ ]:
def rattle_shake(sec=20, rest=0.4):
    """
    This function is a primitive. When called, the robotic arm will move between two set point in space, defined by the angles the motors are travelling to.
    This will make the arm perform a vertical movement in order to imitate an infant shaking a rattle.

        :param int sec: time in seconds for how long the arm will wave
            ::default is set to 20 seconds
        :param float rest: time in seconds for the time the arm has to finish its motion
            ::default is set to 0.4 seconds
    """
    
    # define the first point in space
    shoulder_y0 = -45
    shoulder_x0 = 0
    arm_0 = 0
    elbow_0 = -65
    
    # define the second point in space
    shoulder_y1 = -30
    shoulder_x1 = 0
    arm_1 = 0
    elbow_1 = -35
    
    # speed adjustments to be more infant like
    poppy.r_shoulder_y.moving_speed = abs(shoulder_y1-shoulder_y0) / rest
    poppy.r_shoulder_x.moving_speed = abs(shoulder_x1-shoulder_x0) / rest
    poppy.r_arm_z.moving_speed = abs(arm_1-arm_0) / rest
    poppy.r_elbow_y.moving_speed = abs(elbow_1-elbow_0) / rest

    
    
    t0 = time.time() # Gets the timestamp at the start of the waving motion
    
    # This will loop through the two arm positions until the set time duration has elapsed. 
    while True:
              
        # arm moves to first position
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = shoulder_y0 #-45
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = shoulder_x0 #5
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = arm_0 #20
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = elbow_0 #-129
        

        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
        
        # arm moves to second position
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = shoulder_y1 #-25
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = shoulder_x1  #0
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = arm_1; #0
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = elbow_1 #-10
        


       
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
        
        
        app()
        TIME.append(t1-t0)
    
         
        t1 = time.time() # Gets the timestamp at the end of each loop
        
        if t1-t0 >= sec: # This will end the loop and wave motion when the allotted time has expired
            break

    
    # resets motor speed
    for x in poppy.motors:
            x.moving_speed = 90

## Resets the arrays and returns the arm to the resting position

Useful for running different motions independently

In [ ]:
rest_position()

pos_Head = []
pos_Stand = []
TIME = []
sys_load = []
speed = []
temp = []
volt = []

In [ ]:
# Run to to make the arm wave
hand_wave()
time.sleep(0.25) # Gives the arm a little time to complete a motion before commencing the next action
rest_position()  # Optional: returns the arm to the resting position

In [ ]:
# Run to make the arm perform a rattle shake movement
ratttle_shake()
time.sleep(0.25) # Gives the arm a little time to complete a motion before commencing the next action
rest_position()  # Optional: returns the arm to the resting position

## Run when experiments are done

This cell resets the robot and releases the robot.
After running this set of commands, rerunning the setup cell is necessary

### **NOTE**
If this fails or some error occurs, it may be necessary to run a hard reset:
    
    For physical robot:
        1. Unplug the robot
    
    For simulator:
        1.Close the simulator
    
    2. Save and shutdown the ipython notebook
    3. It may be a good idea to also close the jupyter-notebook server and restart from there.

In [ ]:
reached_pt_Stand = []
reached_pt_Head = []
for m in poppy.motors:
    m.compliant = True
time.sleep(2)
poppy.stop_sync()
poppy.close()